In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
import numpy as np
import csv

# Read CSV
df = pd.read_csv('./Comments Upload.csv')

# Change comment to string type
comment_array = np.array([str(i) for i in df['Result'].values])
# Transform CSV to List
comment_list = list(comment_array)

# Call NLP Token
tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
model = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english")

# Sentiment Analysis
inputs = tokenizer(comment_list, padding = True, truncation = True, return_tensors='pt')
outputs = model(**inputs)

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Tranform result in CSV
positive = predictions[:, 1].tolist()
negative = predictions[:, 0].tolist()

table = {'Comment':comment_list,
         "Positive":positive,
         "Negative":negative}
      
new_df = pd.DataFrame(table, columns = ['Comment', "Positive", "Negative"])

#Create new column 'Attitude' for result conclustion
new_df['Attitude'] = new_df[["Positive", "Negative"]].idxmax(axis=1)

# Call second NLP Token
tokenizer1 = AutoTokenizer.from_pretrained("NCHS/SANDS")
model1 = AutoModelForSequenceClassification.from_pretrained("NCHS/SANDS")
inputs1 = tokenizer1(comment_list, padding = True, truncation = True, return_tensors='pt')
outputs1 = model1(**inputs1)

predictions1 = torch.nn.functional.softmax(outputs1.logits, dim=-1)

# Tranform 2nd result in CSV
Refusal = predictions1[:, 0].tolist()
High_risk = predictions1[:, 1].tolist()
Gibberish = predictions1[:, 2].tolist()
Uncertainty = predictions1[:, 3].tolist()
Valid = predictions1[:, 4].tolist()

table1 = {'Comment':comment_list,
         "Refusal":Refusal,
         "High-risk":High_risk,
         "Gibberish":Gibberish,
         "Uncertainty":Uncertainty,
         "Valid":Valid}
      
new_df1 = pd.DataFrame(table1, columns = ['Comment', "Refusal","High-risk","Gibberish","Uncertainty","Valid"])

#Create new column 'Vaild?' for valid conclustion
new_df1['Vaild?'] = new_df1[["Refusal","High-risk","Gibberish","Uncertainty","Valid"]].idxmax(axis=1)

#Combine 2 results
results = pd.concat([new_df, new_df1], axis=1)

#Conclude the result
results['Results record'] = np.where(results['Vaild?'] != "Valid", "Invalid", results['Attitude'])

results.to_csv("PCCW Commets Testing.csv", index = False)